In [1]:
import pandas as pd

import __init__

from db.pgdf import PgDf
from db.postgresql import PostgreSqlDB

d = PostgreSqlDB()
pg = PgDf()

sql = """WITH r AS (
SELECT
    codename,
    dt,
    RANK() OVER (PARTITION BY date_trunc('month', dt) ORDER BY v DESC) as rank
FROM 
    ohlcv_mo1
WHERE
    codename NOT LIKE '^%%' 
    AND codename NOT IN ('1357.T', '1570.T')
    AND codename IN ('1757.T')
)
select codename, dt from r
WHERE
    rank <= 50
"""

df = None
for (codename, dt) in d.execSql(sql):
    sql = """WITH g as (
SELECT 
    codename, 
    date_trunc('month', dt) as month, 
    ROUND(CAST(LEFT(CAST(price AS TEXT), 4) AS FLOAT) / 10) as grp
FROM peaks_d_10
WHERE
    codename = '%s'
    AND dt < '%s'::date + interval '1 month'
    AND dt >= '%s'::date - interval '1 year'
), gsize as (
SELECT codename, month, grp, count(*) as grp_size from g
GROUP BY 1,2,3
HAVING count(*) > 1
)
SELECT codename, grp_size, count(*) as cnt from gsize
GROUP BY 1,2
;
""" % (codename, dt, dt)
    tmp = pg.read_sql(sql)
    if df is None:
        df = tmp
    else:
        df = pd.concat([df, tmp], ignore_index=True)

print("completed")




completed


In [2]:
summary_df = df.groupby('grp_size')['cnt'].sum().reset_index()
summary_df.sort_values(by='cnt', ascending=False)
summary_df

,grp_size,cnt
0,2,284
1,3,286
2,4,139
3,5,59
4,6,39
5,8,13
6,9,13
7,10,13
8,11,34
9,12,13
